In [50]:
import os
import rasterio
from rasterio.enums import Resampling
from rasterio.shutil import copy

def resample_directory(directory, resolution=0.04, compression='LZW'):
    # Get all GeoTIFF files in the directory
    files = [f for f in os.listdir(directory) if f.endswith('.tif')]

    for file in files:
        with rasterio.open(os.path.join(directory, file)) as src:
            # Calculate the new shape based on the new resolution
            new_shape = (int(src.shape[0] * src.res[0] / resolution),
                         int(src.shape[1] * src.res[1] / resolution))

            # Resample the raster
            data = src.read(
                out_shape=new_shape,
                resampling=Resampling.bilinear
            )

            # Update the metadata
            transform = src.transform * src.transform.scale(
                (src.width / data.shape[-1]),
                (src.height / data.shape[-2])
            )

            profile = src.profile
            profile.update(
                dtype=rasterio.float32,
                height=new_shape[0],
                width=new_shape[1],
                transform=transform,
                compress=compression
            )

            # Save the resampled raster
            with rasterio.open(os.path.join(directory, f'resampled_{file}'), 'w', **profile) as dst:
                dst.write(data)

In [51]:
resample_directory("G:/My Drive/teaching/2024_msc_remotesensing_geoinformatics/data/photogrammetric_drone_products/confobi/")